In [1]:
import time
import urllib
import requests
import platform
import matplotlib.pyplot as plt

from platform import platform
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud

In [2]:
def getText(search_kwrd, page_num=1, text=''):
	"""html page에서 text만 가져오는 함수"""
	# requests settings
	session = requests.Session()
	headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)'\
			   'AppleWebKit 537.36 (KHTML, like Gecko) Chrome',\
			   'Accept': 'text/html, application/xhtml+xml, application/xml;'\
			   'q=0.9, image/webp, */*; q=0.8'}
	
	# url에 있는 한글 encoding 
	search_kwrd_lst = search_kwrd.split()
	keyword = ''
	for k in search_kwrd_lst:
		keyword = keyword + urllib.parse.quote(k) + '+'
	keyword = keyword.rstrip('+')

	# scraping
	bs = []
	for n in range(1, page_num+1):
		req = session.get(f'https://search.daum.net/search?p={n}&q={keyword}&w=fusion&DA=PGD&period=6m&sd=20220126215757&ed=20220726215757',\
						  headers=headers)
		time.sleep(1)
		bsobj = BeautifulSoup(req.text, 'html.parser')

		# Daum 통합웹 탭에서 해딩 키워드 검색 데이터
		bsobj = bsobj.find('ul', class_='list_info ty_doc')  
		bs.append(bsobj)

	# html tag를 제외한 text만 추출
	for page in bs:
		page = page.text
		text += page.replace(' ', '')
		for i in range(len(search_kwrd_lst)):
			text += text.replace(search_kwrd_lst[i], '')
	
	return text


def mk_wordcloud(text):
	"""WordCloud 생성 함수"""
	okt = Okt()  # Open Korean Text 객체 생성

	# 명사와 형용사만 noun_adj_list에 추가
	sentences_tag = []
	sentences_tag = okt.pos(text)
	noun_adj_list = []
	for word, tag in sentences_tag:
		if tag in ['Noun' , 'Adjective']:
			noun_adj_list.append(word)

	# 가장 많이 나온 단어 40개 저장
	counts = Counter(noun_adj_list)
	tags = counts.most_common(40)

	# 한글 분석을 위한 font path 설정
	if platform.system() == 'Windows':
		path = r'c:\Windows\Fonts\malgun.ttf'
	elif platform.system() == 'Darwin':
		path = r'/System/Library/Fonts/AppleGothic'
	else:
		path = r'/usr/share/fonts/truetype/name/NanumMyeongjo.ttf'

	# WordCloud 생성
	wc = WordCloud(font_path=path, background_color="white", max_font_size=50)
	cloud = wc.generate_from_frequencies(dict(tags))

	# WordCloud 출력
	cloud.to_file('guam_sipan.png')
	plt.figure(figsize=(10, 10))
	plt.axis('off')
	plt.imshow(cloud)
	plt.show()

In [3]:
if __name__ == '__main__':
	keyword = '괌 사이판 여행'  # 검색 키워드
	text = getText(search_kwrd=keyword, page_num=3)  # 1~5페이지 scraping
	mk_wordcloud(text)